## Python Imports

In [ ]:
!pip install accelerate
!pip install transformers
!pip install datasets
!pip install bert_score
!pip install evaluate
!pip install nltk

import accelerate
import torch
import pandas as pd
import numpy as np

import os
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, GenerationConfig,  AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

import nltk
nltk.download('punkt_tab')
nltk.download('punkt')   # Ensure 'punkt' is downloaded
from evaluate import load

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('/content/tqsk3.csv')
tdf = df.iloc[:77]
vdf = df.iloc[77:]

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
checkpoint = "gpt2-medium"  # Use valid GPT-2 checkpoints like gpt2, gpt2-medium, gpt2-large, or gpt2-xl
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
# gen_config = GenerationConfig.from_pretrained(checkpoint)

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
# def tokenize_function(example):
#     inputs = tokenizer(example['Prompt'], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
#     with tokenizer.as_target_tokenizer():
#         targets = tokenizer(example['Explanation'], max_length=80, truncation=True, padding="max_length", return_tensors="pt")

#     inputs['labels'] = targets['input_ids']
#     return inputs

# # df['Prompt'] = np.where(df['Narrative 2'] != 'none',
# #                         'Based on the narrative following narrative: ' + df['Narrative 2']+ '  find the summary of the: ' + df['Content'],
# #                         'Based on the narrative following narrative: ' + df['Narrative 1']+ '  find the summary of the: ' + df['Content'])

# df_train = tdf[['Prompt', 'Explanation']].copy()
# df_val = vdf[['Prompt', 'Explanation']].copy()

# traindataset = Dataset.from_pandas(df_train)
# train_dataset = traindataset.map(tokenize_function, batched=True)

# validationdataset = Dataset.from_pandas(df_val)
# validation_dataset = validationdataset.map(tokenize_function, batched=True)




from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from datasets import Dataset

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # To handle edge cases where padding is still needed

def tokenize_function(examples):
    # Concatenate the prompt and explanation for each example in the batch
    input_texts = [
        prompt + " " + tokenizer.eos_token + " " + explanation
        for prompt, explanation in zip(examples['Prompt'], examples['Explanation'])
    ]

    # Tokenize the list of concatenated input-output sequences
    tokenized = tokenizer(
        input_texts,
        truncation=True,
        max_length=1024,  # Adjust max_length based on your use case
        padding=False  # Padding will be handled later dynamically
    )

    # Set labels for language modeling
    tokenized['labels'] = tokenized['input_ids'].copy()

    return tokenized

# Prepare training and validation data
df_train = tdf[['Prompt', 'Explanation']].copy()
df_val = vdf[['Prompt', 'Explanation']].copy()

# Convert DataFrames to datasets
traindataset = Dataset.from_pandas(df_train)
validationdataset = Dataset.from_pandas(df_val)

# Tokenize the datasets
train_dataset = traindataset.map(tokenize_function, batched=True, remove_columns=traindataset.column_names)
validation_dataset = validationdataset.map(tokenize_function, batched=True, remove_columns=validationdataset.column_names)

# Custom data collator for variable-length batching
class DataCollatorForDynamicBatching:
    def __call__(self, features):
        # Flatten all examples into input_ids and labels
        input_ids = [feature['input_ids'] for feature in features]
        labels = [feature['labels'] for feature in features]

        # Dynamically pad sequences in the batch
        batch_input_ids = tokenizer.pad(
            {"input_ids": input_ids},
            padding=True,
            return_tensors="pt",
        )['input_ids']

        batch_labels = tokenizer.pad(
            {"input_ids": labels},
            padding=True,
            return_tensors="pt",
        )['input_ids']

        return {"input_ids": batch_input_ids, "labels": batch_labels}

data_collator = DataCollatorForDynamicBatching()

# Example for DataLoader
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
validation_loader = DataLoader(validation_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)


Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
df['Prompt'] = df['Prompt'].astype(str)
print(df['Prompt'].dtype)
print(df['Prompt'].head())

object
0    Based on the narrative following narrative: UR...
1    Based on the narrative following narrative: UR...
2    Based on the narrative following narrative: UR...
3    Based on the narrative following narrative: CC...
4    Based on the narrative following narrative: UR...
Name: Prompt, dtype: object


In [ ]:
# Load the metrics using evaluate
bertscore = load("bertscore")

def compute_metrics(pred):
    pred_logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in pred_ids]

    # Remove ignored index (-100) from labels and replace with pad token id
    labels_ids = np.where(pred.label_ids != -100, pred.label_ids, tokenizer.pad_token_id)
    label_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels_ids]

    # Compute ROUGE score
    bertscore_output = bertscore.compute(predictions=pred_str, references=label_str, lang="en")
    # Return ROUGE and BLEU scores
    return {
        'bertscore_precision': sum(bertscore_output['precision']) / len(bertscore_output['precision']),
        'bertscore_recall': sum(bertscore_output['recall']) / len(bertscore_output['recall']),
        'bertscore_f1': sum(bertscore_output['f1']) / len(bertscore_output['f1']),
    }


# Initialize WandB with disabled mode
import wandb
wandb.init(mode="disabled")


In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',           # Directory to save checkpoints
    num_train_epochs=3,               # Adjust epochs as per resources and dataset size
    per_device_train_batch_size=2,    # Reduce batch size if you encounter memory issues
    per_device_eval_batch_size=2,
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=50,                 # Log every X updates
    save_strategy="epoch",            # Save model after each epoch
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
    load_best_model_at_end=True,      # Load the best model found during evaluation
    learning_rate=5e-5,               # Learning rate
    save_total_limit=3,               # Limit the number of saved checkpoints
    metric_for_best_model="bertscore_f1",          # Use loss to determine the best model
    greater_is_better=True,          # Lower loss is better
    report_to="none",                 # Disable reporting to third-party platforms like W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,  # Handles padding dynamically
    compute_metrics=compute_metrics
    )

trainer.train()
# 72ec5df7b7871e699a9c5e61b91a0cb74038b3c1

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-31-95e72a3b86ac>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=117, training_loss=2.8962787073901577, metrics={'train_runtime': 365.7686, 'train_samples_per_second': 0.632, 'train_steps_per_second': 0.32, 'total_flos': 366880307183616.0, 'train_loss': 2.8962787073901577, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")

del model
torch.cuda.empty_cache()

model = GPT2LMHeadModel.from_pretrained("/content/")
tokenizer = GPT2Tokenizer.from_pretrained("/content/")
generation_config = GenerationConfig.from_pretrained("/content/")

In [ ]:

# generation_config = GenerationConfig(
#     max_length=150,
#     min_length=56,
#     # early_stopping=True,
#     early_stopping=False,
#     num_beams=4,
#     length_penalty=2.0,
#     no_repeat_ngram_size=3,
#     forced_bos_token_id=0,
#     forced_eos_token_id=2
# )

# Load the model and tokenizer
# model = AutoModelForCausalLM.from_pretrained("/content/")
# tokenizer = AutoTokenizer.from_pretrained("/content/")
# generation_config = GenerationConfig.from_pretrained("/content/")

In [ ]:
# tedf

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)


# def summarize_text(text):

#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
#     summary_ids = model.generate(
#         inputs['input_ids'],
#         generation_config=generation_config
#     )
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# # tedf['summary'] = tedf['Content'].apply(summarize_text)
# # print(tedf['Content'])
# # tedf['summary'] = tedf['Prompt'].apply(summarize_text)

In [ ]:
# predictions = tedf['summary'].tolist()
# references = tedf['Explanation'].tolist()
# # results = bertscore.compute(predictions=predictions, references=references, lang="en")
# results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
# results

In [ ]:
# predictions

In [ ]:
# references

# THIS IS ADDED LATER AFTER THE CODE WORKED SO PLS DONT CHANGE ABOVE THIS

In [ ]:
import requests

# Raw URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-3-dominant-narratives.txt"

# Load the annotations file
df1 = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'Narrative 1', 'Narrative 2'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-3-documents/"

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = f"{base_url}{article_id}"
    # print(f"Fetching URL: {file_url}")  # Debugging URL
    response = requests.get(file_url)
    # print(f"Status Code: {response.status_code}")  # Debugging response status
    if response.status_code == 200:
        return response.text
    else:
        return f"File {article_id} not found"


# Fetch article content for each article_id
df1['Content'] = df1['article_id'].apply(fetch_file_content)



# Generate the Prompt column
df1['Prompt'] = np.where(
    df1['Narrative 2'] != 'none',
    'Based on the the: ' + df1['Narrative 2'] + ' Give the summary of the: ' + df1['Content'],
    'Based on the the: ' + df1['Narrative 1'] + ' Give the summary of the: ' + df1['Content'],
)

ledf = df1.copy()

In [ ]:
ledf['Prompt'] = ledf['Prompt'].apply(lambda x: x[:4290] if isinstance(x, str) else x)
print(ledf['Prompt'].apply(len).max())
ledf

4290


article_id                                        Narrative 1  \
0   EN_UA_DEV_100012.txt              URW: Discrediting the West, Diplomacy   
1       EN_CC_200040.txt                  CC: Criticism of climate movement   
2       EN_CC_200070.txt      CC: Criticism of institutions and authorities   
3   EN_UA_DEV_100034.txt                         URW: Overpraising the West   
4       EN_CC_200049.txt       CC: Questioning the measurements and science   
5   EN_UA_DEV_100033.txt                      URW: Speculating war outcomes   
6       EN_CC_200036.txt                  CC: Criticism of climate movement   
7       EN_CC_200079.txt      CC: Criticism of institutions and authorities   
8   EN_UA_DEV_100029.txt                      URW: Speculating war outcomes   
9       EN_UA_DEV_20.txt                          URW: Discrediting Ukraine   
10  EN_UA_DEV_100005.txt              URW: Discrediting the West, Diplomacy   
11      EN_CC_200033.txt                  CC: Criticism of climate movement   
12      EN_CC_200071.txt                  CC: Criticism of climate policies   
13      EN_CC_200046.txt                   CC: Climate change is beneficial   
14  EN_UA_DEV_100026.txt            URW: Negative Consequences for the West   
15     EN_UA_DEV_215.txt                          URW: Discrediting Ukraine   
16      EN_CC_200054.txt  CC: Hidden plots by secret schemes of powerful...   
17  EN_UA_DEV_100013.txt                          URW: Discrediting Ukraine   
18      EN_CC_200077.txt                  CC: Criticism of climate policies   
19      EN_CC_200034.txt                  CC: Criticism of climate movement   
20     EN_UA_DEV_213.txt  URW: Blaming the war on others rather than the...   
21      EN_CC_200047.txt  CC: Hidden plots by secret schemes of powerful...   
22      EN_CC_200081.txt           CC: Controversy about green technologies   
23     EN_UA_DEV_214.txt              URW: Discrediting the West, Diplomacy   
24      EN_CC_200064.txt                  CC: Criticism of climate movement   
25      EN_UA_DEV_22.txt              URW: Discrediting the West, Diplomacy   
26      EN_CC_200035.txt      CC: Criticism of institutions and authorities   
27      EN_CC_200069.txt                  CC: Criticism of climate policies   
28      EN_CC_200065.txt                  CC: Criticism of climate movement   
29  EN_UA_DEV_100002.txt                      URW: Speculating war outcomes   

                                          Narrative 2  \
0   URW: Discrediting the West, Diplomacy: The Wes...   
1                                                none   
2   CC: Criticism of institutions and authorities:...   
3   URW: Overpraising the West: The West belongs i...   
4                                                none   
5   URW: Speculating war outcomes: Russian army is...   
6   CC: Criticism of climate movement: Ad hominem ...   
7   CC: Criticism of institutions and authorities:...   
8   URW: Speculating war outcomes: Russian army is...   
9   URW: Discrediting Ukraine: Situation in Ukrain...   
10                                               none   
11                                               none   
12  CC: Criticism of climate policies: Climate pol...   
13  CC: Climate change is beneficial: CO2 is benef...   
14  URW: Negative Consequences for the West: Sanct...   
15  URW: Discrediting Ukraine: Discrediting Ukrain...   
16  CC: Hidden plots by secret schemes of powerful...   
17  URW: Discrediting Ukraine: Discrediting Ukrain...   
18  CC: Criticism of climate policies: Climate pol...   
19  CC: Criticism of climate movement: Climate mov...   
20  URW: Blaming the war on others rather than the...   
21  CC: Hidden plots by secret schemes of powerful...   
22  CC: Controversy about green technologies: Rene...   
23  URW: Discrediting the West, Diplomacy: The Wes...   
24                                               none   
25  URW: Discrediting the West, Diplomacy: The EU ...   
26  CC: Criticism of inst

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # Set `pad_token_id` to `eos_token_id`
# if model.config.pad_token_id is None:
#     model.config.pad_token_id = model.config.eos_token_id

# # Summarization function
# def summarize_text(text):
#     # Tokenize the input with attention mask
#     inputs = tokenizer(
#         text,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=1024
#     ).to(device)  # Move tensors to the same device as the model

#     # Generate summary with explicit generation config
#     summary_ids = model.generate(
#         inputs['input_ids'],
#         attention_mask=inputs['attention_mask'],  # Provide attention mask
#         max_length=1024,
#         pad_token_id=model.config.pad_token_id  # Explicitly set pad_token_id
#     )

#     # Decode the generated summary
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# # Apply summarization to the DataFrame
# ledf['summary'] = ledf['Prompt'].apply(summarize_text)

KeyError: 'Prompt'

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)


# def summarize_text(text):

#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length = 1024).to(device)
#     summary_ids = model.generate(
#         inputs['input_ids'],
#         max_length=1024,
#         generation_config=generation_config
#     )
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# ledf['summary'] = ledf['Prompt'].apply(summarize_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
ledf['summary']
print(ledf['summary'].apply(len).max())
ledf.drop(columns=['Narrative 1','Narrative 2','Content', 'Prompt'], inplace=True)

5131


In [ ]:
output_txt_path = "mums.txt"  # Replace with the desired TXT file path
ledf.to_csv(output_txt_path, sep="\t", index=False, header=False)